In [2]:
sc.stop()

NameError: name 'sc' is not defined

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 5 --executor-memory 4g --executor-cores 1 --driver-memory 3g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "PGG-try2") 

spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.jp-OutputArea-output pre {white-space: pre !important; }</style>"))

In [5]:
from pyspark.mllib.linalg import SparseVector, DenseVector
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover,VectorAssembler
import pyspark.sql.functions as f_
# from pyspark.sql.types import FloatType, DoubleType, DecimalType, StringType, ArrayType 
from pyspark.sql.types import FloatType
from pyspark.sql.window import Window
from pyspark.ml import Pipeline
import json

In [6]:
@f_.udf(FloatType())
def cosine_similarity(v,u):
    return float(v.dot(u) / (v.norm(2) * u.norm(2)))


In [7]:
personal_conditions=[[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
pc_ids=[p[0] for p in personal_conditions]
set([p[1] for p in personal_conditions])

{'en', 'es', 'ru'}

In [8]:
regexp_clear= f_.lower(f_.regexp_replace('desc',r'[^\pL0-9\p{Space}]','' ))

In [9]:
stop_words = list(set(
    StopWordsRemover.loadDefaultStopWords("english") + 
    StopWordsRemover.loadDefaultStopWords("russian") + 
    StopWordsRemover.loadDefaultStopWords("spanish")))

In [10]:
# Загружаем данные 
data = spark.read.json('/labs/slaba02/DO_record_per_line.json')
# Чистом данные
courses_data=data.select(*data.columns ,regexp_clear.alias('doc')).drop('desc')

In [11]:
tokenizer = Tokenizer(inputCol="doc", outputCol="words")
swr=StopWordsRemover(inputCol="words", outputCol="words_censored", stopWords=stop_words)
tf = HashingTF(inputCol="words_censored", outputCol="tf")
# hashingTF = HashingTF(numFeatures =10000, inputCol="words", outputCol="tf")
tfidf = IDF(inputCol="tf", outputCol="idf")
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    tf,
    tfidf
])

In [12]:
calc=pipeline.fit(courses_data)
tfidf_data=calc.transform(courses_data)

In [13]:
cross=tfidf_data.alias('a').crossJoin(tfidf_data.where(f_.col('id').isin(pc_ids)).alias('b'))\
.select('a.*'
        ,f_.col('b.id').alias('pc_id')
        ,f_.col('b.lang').alias('pc_lang')
        ,f_.col('b.name').alias('pc_name')
        ,f_.col('b.idf').alias('pc_tfidf')
       )\
.filter(''' a.lang=pc_lang and pc_id !=id ''')

In [14]:
cross2=cross.select('*',cosine_similarity('idf','pc_tfidf').alias('cos'))\
.filter(''' cos !='NaN' ''')

In [15]:
cross3=cross2.select('*', f_.row_number().over(Window.partitionBy("pc_id")\
.orderBy(
     f_.col('cos').desc()
    ,f_.col('name')
    ,f_.col('id'))).alias('n'))\
.filter('n<=10')

In [16]:
recomendation_=cross3.groupBy(f_.col('pc_id')).agg(f_.collect_list(f_.col('id')).alias('top_10_ids')).orderBy('pc_id').collect()
recomendation={}
for r in recomendation_:
    recomendation[str(r[0])]=r[1]
    print(str(r[0]),':',r[1])

11556 : [16488, 468, 13461, 22710, 23357, 10447, 19330, 21707, 11523, 9465]
13702 : [864, 21079, 8123, 1041, 28074, 13057, 1396, 1052, 1033, 8300]
16627 : [11431, 12247, 12660, 5687, 17964, 16694, 12598, 11575, 12863, 21704]
16704 : [1236, 1247, 1228, 1365, 1164, 1233, 1273, 20288, 8186, 8203]
21617 : [21609, 21608, 21616, 21492, 21624, 21623, 21703, 21630, 21628, 21508]
23126 : [14760, 13665, 13782, 15909, 19270, 25782, 17499, 13348, 7153, 25071]


In [17]:
with open('lab02.json', 'w') as fp:
    json.dump(recomendation, fp)